In [87]:
import pandas as pd
import numpy as np

In [9]:
# percent cover data
df_pc = pd.read_table('relevant/percent_cover_zipcode.txt', delimiter = ',')
df_pc.columns = ['zip', 'pc_cover']
df_pc_nyc = df_pc.loc[(df_pc['zip'] >= 10001) & (df_pc['zip'] <= 11697)]

In [21]:
df_cities_zips = pd.read_csv('relevant/us_cities_zips.csv')
# nyc zip codes: 10001 to 11697
df_zip_nyc = df_cities_zips.loc[(df_cities_zips['zip_code'] >= 10001) & (df_cities_zips['zip_code'] <= 11697)]
df_zip_nyc_relevant = df_zip_nyc[['zip_code', 'population', 'density']]
df_zip_nyc_relevant.columns = ['zip', 'population', 'density']

In [30]:
# merge datasets
combined = pd.merge(df_zip_nyc_relevant, df_pc_nyc, how="outer")
combined = combined.dropna()

In [95]:
# covid data
df_hosprate = pd.read_csv('relevant/covid/hosprate-by-modzcta.csv')
df_deathrate = pd.read_csv('relevant/covid/deathrate-by-modzcta.csv')
df_percentpos = pd.read_csv('relevant/covid/percentpositive-by-modzcta.csv')

# df_hosprate = df_hosprate.transpose()
# df_hosprate.columns = df_hosprate.iloc[0]
# df_hosprate.drop(df_hosprate.index[0])

In [130]:
# get covid stats [hosp rate, death rate, percent pos] averaged by zip code across all months in 2020-current
temp = []
cols = df_hosprate.columns
for i in range(7, len(cols)):
    hosp_avg = np.nanmean(list(df_hosprate[df_hosprate.columns[i]]))
    death_avg = np.nanmean(list(df_deathrate[df_deathrate.columns[i]]))
    percentpos_avg = np.nanmean(list(df_percentpos[df_percentpos.columns[i]]))
    temp.append([cols[i][-5:], hosp_avg, death_avg, percentpos_avg])
    
covid_data = pd.DataFrame(data = temp, columns = ['zip', 'hosp_avg', 'death_avg', 'percentpos_avg'])
covid_data['zip']=covid_data['zip'].astype(int)

In [133]:
df_combined = combined.merge(covid_data, left_on='zip', right_on='zip')

In [137]:
df_combined

,zip,population,density,pc_cover,hosp_avg,death_avg,percentpos_avg
0,10001,25026.0,15489.3,2.612,37.754545,8.792857,5.624769
1,10002,74363.0,34916.7,11.836,71.197143,27.155000,7.029615
2,10003,54671.0,36928.8,6.273,27.411765,8.000000,5.488692
3,10004,3310.0,2805.2,24.690,67.271429,0.000000,5.929385
4,10005,8664.0,45907.6,2.109,28.066667,0.000000,5.970538
...,...,...,...,...,...,...,...
172,11691,68454.0,9849.6,22.408,82.045714,48.810526,7.458692
173,11692,22394.0,8511.5,22.596,68.820588,33.050000,7.366923
174,11693,13029.0,5088.1,34.662,62.324138,21.038889,7.904692
175,11694,21258.0,4697.4,3.773,67.843333,33.993750,7.152692


In [136]:
df_combined.corr(method='pearson')

,zip,population,density,pc_cover,hosp_avg,death_avg,percentpos_avg
zip,1.000000,0.129937,-0.497263,-0.089982,0.253898,0.378413,0.438812
population,0.129937,1.000000,0.167139,-0.162176,0.193378,0.326682,0.006114
density,-0.497263,0.167139,1.000000,-0.198399,-0.356450,-0.288728,-0.475898
pc_cover,-0.089982,-0.162176,-0.198399,1.000000,0.109307,-0.022119,0.080533
hosp_avg,0.253898,0.193378,-0.356450,0.109307,1.000000,0.745045,0.446150
death_avg,0.378413,0.326682,-0.288728,-0.022119,0.745045,1.000000,0.328625
percentpos_avg,0.438812,0.006114,-0.475898,0.080533,0.446150,0.328625,1.000000
